<a href="https://colab.research.google.com/github/Hongbi-Kim/Writing_lyrics/blob/master/220428_html_fastapi_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코랩이 구글 드라이브에 마운트돼 있어야 합니다.

구글드라이브에 front-end 숏컷이 만들어져 있어야 합니다.

결과 표시를 위해서 txt파일로 저장하는 명령을 추가했습니다.

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 14.7 MB/s 
     |████████████████████████████████| 596 kB 76.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.6 MB/s 
     |████████████████████████████████| 895 kB 92.9 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

     |████████████████████████████████| 19.4 MB 15.0 MB/s 
     |████████████████████████████████| 448 kB 66.9 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-05-01 03:27:47--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
L

In [4]:
from collections import Counter

In [5]:
# 구글 드라이브에서 모델, 토크나이저 불러오기
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)


from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

# gpt3-kor-small_based_on_gpt2
from transformers import BertTokenizerFast, GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/gpt3_topic')
tokenizer_gpt3 = BertTokenizerFast.from_pretrained('/content/drive/MyDrive/gpt3_topic')
input_ids = tokenizer_gpt3.encode("text to tokenize")[1:]  # remove cls token

In [ ]:
#from konlpy.tag import Okt
#okt = Okt()

In [6]:
from konlpy.tag import Mecab
mecab = Mecab()

In [7]:
device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [8]:
model.eval()
def lyric_generator(gen, lyric):
  prompt = f"<|startoftext|> <{gen}> {lyric}"
  generated = torch.tensor(tokenizer_gpt3.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 350,
                                  #max_length = len(lyric) + 350,
                                  top_p=0.95, 
                                  num_return_sequences=1,
                                  repetition_penalty=1.1
                                  )
  for i, sample_output in enumerate(sample_outputs):
    # result = "{}: {}".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
    result = tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)
    result = re.sub(r"<br>", '\n', result)
    lyrictxt = open(r'/content/drive/MyDrive/front-end/lyrictxt.txt','w')
    myString = result
    lyrictxt.write(myString)
    lyrictxt.close()
    # re.sub(r'[a-z]','',result.split('\n')[-1])
    # result = re.sub(r'[a-z]','',result.split('\n')[-1])   
    # result = result[len(lyric)+1:]
    # print(re.sub(r'[a-z]','',result))
    print(result)
  return result
  # for i, sample_output in enumerate(sample_outputs):
  #   # result = "{}: {}".format(i, tokenizer_gpt3.decode(sample_output, skip_special_tokens=True))
  #   result = tokenizer_gpt3.decode(sample_output, skip_special_tokens=True)
  #   result = re.sub(r"<br>", '\n', result)
  #   # re.sub(r'[a-z]','',result.split('\n')[-1])
  #   # result = re.sub(r'[a-z]','',result.split('\n')[-1])   
  #   # result = result[len(lyric)+1:]
  #   # print(re.sub(r'[a-z]','',result))
  #   print(result)
  # return result

In [9]:
def decide_topic(gen, lyric):

  d_topic1 = ['밤','심장','순간','날','눈','몸','숨','눈빛','머리','시선','맘','끝','손','기분','춤','시작','입술','원','준비','소리',
              '음악','느낌','때','완벽','안','위','필요','리듬','입','전']
  d_topic2 = ['눈','밤','속','순간','꿈','손','하늘','날','위','끝','숨','세상','맘','심장','시간','빛','바람','눈빛','안','기분','소리',
              '시작','별','길','때','몸','곳','노래','향기','시선']
  d_topic3 = ['남자','집','여자','매력','친구','옷','머리','생일','기분','전화','축하','노래','화장','밥','티','커피','눈치','정신','아침',
              '스타일','오늘','애','문자','몸매','분위기','얼굴','영화','오빠','번호','생일날']
  d_topic4 = ['엄마','돈','아빠','인생','나이','공부','아저씨','집','왕','애','학교','금','배','물','개','아이','맛','키','형','밥','산타',
              '오빠','몫','랩','산','동생','부모','술','라면','남']
  d_topic5 = ['춤','몸','음악','머리','인생','볼륨','준비','리듬','소리','다리','무대','돈','잔','비트','걱정','남','승리','젋음','발','폼',
              '모두','허리','땀','함성','해변','엉덩이','차','피해','게임','해결']

  b_topic1 = ['꿈','속','하늘','빛','위','끝','세상','순간','길','별','손','어둠','숨','곳','시간','시작','바람','안','눈','밤','미래','태양',
              '현실','구름','아래','날','달','내일','달빛','삶']
  b_topic2 = ['사랑','말','날','마음','생각','맘','사람','시간','모습','속','때','가슴','행복','곁','눈물','기억','세상','하루','이상','일','눈',
              '오늘','앞','끝','여자','필요','추억','친구','혼자','밤']
  b_topic3 = ['돈','집','엄마','친구','인생','나이','밥','아빠','술','오빠','차','키','남','여자','부모','잔','학교','동네','아저씨','크리스마스',
              '라면','한잔','결혼','공부','우정','애','모두','언니','배','누나']
  b_topic4 = ['춤','음악','바다','리듬','위','파도','여름','노래','몸','소리','밤','아래','태양','바람','하늘','조명','도시','볼륨','햇살','분위기',
              '기분','여행','모래','파티','걱정','일상','불빛','무대','산','해']
  b_topic5 = ['눈','맘','날','눈빛','밤','기분','심장','머리','손','순간','말','입술','시선','향기','몸','느낌','입','때','원','숨','상상','남자',
              '눈치','매력','준비','완벽','오늘','전','미소','표정']

  token = mecab.pos(lyric)
  #token = okt.pos(lyric)
  #print(token)

  word_list = []
  for t in token:
    if t[1] == 'NNG':
    #if t[1] == 'Noun':
      word_list.append(t[0])
  #print(word_list)
  
  topic_counts = []
  # Counter().moset_common(n)의 경우 같은 수의 카운트는 발생한 순으로 반환
  # 댄스 1-2-3-4-5
  if gen == '댄스':
    for word in word_list:
      if word in d_topic1:
        topic_counts.append('topic1')
      if word in d_topic2:
        topic_counts.append('topic2')
      if word in d_topic3:
        topic_counts.append('topic3')
      if word in d_topic4:
        topic_counts.append('topic4')
      if word in d_topic5:
        topic_counts.append('topic5')

  # 발라드 1-2-3-5-4
  elif gen == '발라드':
    for word in word_list:
      if word in b_topic1:
        topic_counts.append('topic1')
      if word in b_topic2:
        topic_counts.append('topic2')
      if word in b_topic3:
        topic_counts.append('topic3')
      if word in b_topic5:
        topic_counts.append('topic5')
      if word in b_topic4:
        topic_counts.append('topic4')  

  if topic_counts == []:
    topic = 'topic6'
  else:
    topic = Counter(topic_counts).most_common(1)[0][0]
  
  print(topic_counts)
  print(Counter(topic_counts).most_common())
  gen = gen + topic  

  return gen, lyric

In [10]:
decide_topic('발라드', '사랑 하늘')

['topic2', 'topic1', 'topic4']
[('topic2', 1), ('topic1', 1), ('topic4', 1)]


('발라드topic2', '사랑 하늘')

In [11]:
def decide_regen(input):
  val = []
  # for i in input.split('<br>'):
  for i in input.split('\n'):
    val.append(str(bool(re.search(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]',i))))

  eng = []
  space = []
  for char in input:
    if re.sub(r'[a-z]','',char) == "":
      eng += char
    elif char == " ":
      space += char
  eng_ratio = len(eng)/(len(input)-len(space))

  if (len(set(val[1:])) == 1) & ('False' in set(val[1:])) == True:
    print((len(set(val[1:])) == 1) & ('False' in set(val[1:])))
    print(input)
    output = 'regenerate'
  elif eng_ratio > 0.2:
    print(eng_ratio)
    print(input)
    output = 'regenerate'
  else:
    print(eng_ratio)
    output = 'pass'
  
  return output

In [12]:
def remove_lastEng(a):
  # 마지막 줄에 영어만 있다면
  if re.sub(r'[a-z\n]','',a.split('\n')[-1]).strip() == "" :
    b = a.split('\n')[:-1]
    c = ""
    for i in b:
      if i == ' ':
        i = re.sub(r" ", "\n \n", i)
      c += i
      c = re.sub(r"  ", "\n", c)
      output = c
  else:
    output = a
  return output

In [13]:
def ai_lyrics(gen, lyric):
  #print(decide_topic(gen,lyric)[0])
  result = lyric_generator(decide_topic(gen,lyric)[0],decide_topic(gen,lyric)[1])

  for _ in range(10):
    if len(result) < 350:
      print('길이 재생성이 필요합니다')
      result = lyric_generator(decide_topic(gen,lyric)[0],decide_topic(gen,lyric)[1])
    else:
      break

  if decide_regen(result) == 'regenerate':
    print('한글 재생성이 필요합니다.')
    #print(gen)
    output = ai_lyrics(gen, lyric)
  else:
    output = remove_lastEng(result)

  print(output)
  return

In [14]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart # 패스트API 설치

# 패스트API 관련 모듈 불러오기
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn

# 사용할 디렉토리 만들기
!mkdir templates
!mkdir static

     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 745 kB 27.9 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 58 kB 7.3 MB/s 
     |████████████████████████████████| 10.9 MB 91.8 MB/s 
     |████████████████████████████████| 79 kB 10.9 MB/s 
     |████████████████████████████████| 58 kB 7.8 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=4bb489cb66f4287774098e4fd9890bd66131b4169f100c7e0123cc7bcc7f384b
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=6fdc9fe887c8b80fe5588b6cda082bb01334274860c24eecbdf6c4de8589b41d
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


In [ ]:
#  <input class="lead mb-5" type="text" id="lyric" name="lyric" value=" 예) 여긴  동화  속  세상  네버랜드 ">

In [15]:
%%writefile templates/input.html
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>너의 이야기를 들려줘</title>
        <!-- Favicon-->
        <link rel="icon" type="image/x-icon" href="./assets/로고.png" />
        <link href="{{ url_for('static', path='/styles.css') }}" rel="stylesheet" />
        <script src="https://use.fontawesome.com/releases/v6.1.0/js/all.js" crossorigin="anonymous"></script>
    </head>
    <body id="page-top">
        <header class="masthead d-flex align-items-center">
            <div class="container px-4 px-lg-5 text-center">
                <h1>AI 작사가 에이나</h1>
                <h3 class="mb-5"><em> </em></h3>
          <form action="/topic" method="post">
            <h4><strong><label for="gen">발라드와 댄스 중 한 가지 장르를 입력해 주세요</label><br></strong></h4><strong>
            <input type="text" id="gen" name="gen" value="발라드"><br>


            <p>&nbsp;</p>
            <h4><label for="lyric">첫 소절, 또는 도입부를 입력해 주세요</label><br></h4><br>
            <div class="input-group flex-nowrap">
              <span class="input-group-text" id="addon-wrapping">🎶</span>
              <input width="500" class="form-control" type="text" id="lyric" name="lyric" value="여긴 동화 속 세상 네버랜드">
            </div>
            <p>&nbsp;</p>
            <input type="submit" class="btn btn-primary btn-xl" value="가사 생성하기">
           
            </strong></form></div><strong>
        </strong></header><strong>

        <section class="content-section bg-light" id="주의사항">
            <div class="container px-4 px-lg-5 text-center">
                <div class="row gx-4 gx-lg-5 justify-content-center">
                    <div class="col-lg-10">
                        <h2>주의사항</h2>
                        <p class="lead mb-5">
                            본 서비스는 K-Digital Trainning 7,8회차 수강생이 만든 것으로 퍼가는 것은 허용되나,<br>가사의 저작권과 관련된 사항은 서비스 이용자 본인에게 책임이 있음을 알려드립니다.
                        </p>
                    </div>
                </div>
            </div>
        </section>

        <section class="content-section bg-primary text-white text-center" id="제공예정">
            <div class="container px-4 px-lg-5">
                <div class="content-section-heading">
                    <h3 class="text-secondary mb-0">comming soon</h3>
                    <h2 class="mb-5">제공 예정인 서비스</h2>
                </div>
                <div class="row gx-4 gx-lg-5">
                    <div class="col-lg-3 col-md-6 mb-5 mb-lg-0">
                        <span class="service-icon rounded-circle mx-auto mb-3"><i class="icon-screen-smartphone"></i></span>
                        <h4><strong>친구와 공유</strong></h4>
                        <p class="text-faded mb-0">카톡으로 친구와 함께 해봐요</p>
                    </div>
                    <div class="col-lg-3 col-md-6 mb-5 mb-lg-0">
                        <span class="service-icon rounded-circle mx-auto mb-3"><i class="icon-pencil"></i></span>
                        <h4><strong>생성가사 수정</strong></h4>
                        <p class="text-faded mb-0">여러분이 생성된 가사를 튜닝 해봐요</p>
                    </div>
                    <div class="col-lg-3 col-md-6 mb-5 mb-md-0">
                        <span class="service-icon rounded-circle mx-auto mb-3"><i class="icon-like"></i></span>
                        <h4><strong>AI가수</strong></h4>
                        <p class="text-faded mb-0">
                            여러분의 가사를 AI가 불러줘요
                        </p>
                    </div>
                    <div class="col-lg-3 col-md-6">
                        <span class="service-icon rounded-circle mx-auto mb-3"><i class="icon-mustache"></i></span>
                        <h4><strong>서비스센터</strong></h4>
                        <p class="text-faded mb-0">불편사항은 시정하겠습니다</p>
                    </div>
                </div>
            </div>
        </section>
        <!-- Scroll to Top Button-->
        <a class="scroll-to-top rounded" href="#page-top"><svg class="svg-inline--fa fa-angle-up" aria-hidden="true" focusable="false" data-prefix="fas" data-icon="angle-up" role="img" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 384 512" data-fa-i2svg=""><path fill="currentColor" d="M352 352c-8.188 0-16.38-3.125-22.62-9.375L192 205.3l-137.4 137.4c-12.5 12.5-32.75 12.5-45.25 0s-12.5-32.75 0-45.25l160-160c12.5-12.5 32.75-12.5 45.25 0l160 160c12.5 12.5 12.5 32.75 0 45.25C368.4 348.9 360.2 352 352 352z"></path></svg><!-- <i class="fas fa-angle-up"></i> Font Awesome fontawesome.com --></a>
        <!-- Bootstrap core JS-->
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js"></script>
        <!-- Core theme JS-->
        <script src="js/scripts.js"></script>
</strong></body>
</html>

Writing templates/input.html


In [ ]:
# %%writefile templates/result.html
# <!DOCTYPE html>
# <html>
# <head>
# <title>너의 이야기를 들려줘</title>
# <body>
#   <h1 style="text-align: center;"><strong>너의 이야기를 들려줘👂</strong></h1>
#   <h1 style="text-align: center;">생성된 가사입니다.😊</h1>
#   <p>&nbsp</p>
#   <h3 style="text-align: center;"><strong>선택 장르 : {{gen}} </strong></h3>
#     <p>&nbsp</p>
#   <h3 style="text-align: center;"><strong>입력한 사연 : {{lyric}} </strong></h3>
#   <p>&nbsp;</p>
#   <h3 style="text-align: center;"><strong>생성된 가사</strong></h3>
#   <p style="text-align: center;"><iframe src="/static/lyrictxt.txt" width="600px" height="500px" ></iframe>
# </body>

# </html>

In [16]:
%%writefile templates/result.html
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no" />
        <meta name="description" content="" />
        <meta name="author" content="" />
        <title>너의 이야기를 들려줘</title>
        <!-- Favicon-->
        <link rel="icon" type="image/x-icon" href="./assets/로고.png" />
        <link href="{{ url_for('static', path='/styles.css') }}" rel="stylesheet" />
        <script src="https://use.fontawesome.com/releases/v6.1.0/js/all.js" crossorigin="anonymous"></script>
    </head>
    <body class="bg-primary">
    <div class="masthead d-flex align-items-center">
      <div class="container px-4 px-lg-5 text-center">
        <div id="layoutAuthentication">
            <div id="layoutAuthentication_content">
                <main>
                    <div class="container">
                        <div class="row justify-content-center">
                                <div class="card shadow-lg border-0 rounded-lg mt-5">
                                    <div class="card-header"><h3 class="text-center font-weight-light my-4">생성된 가사입니다.</h3></div>
                                    <div class="card-body">
                                        <form>
                                            <div class="form-floating mb-6">
                                                <p style="text-align: center;"><iframe src="/static/lyrictxt.txt" text-align:'center' width="500px" height="500px"></iframe>
                                            </div>
                                        </form>
                                    </div>
                                    <div class="card-footer text-center py-3">
                                        <div class="small"><a class="btn btn-primary" onclick="history.back()">첫 페이지로</a></div>
                                    </div>
                                </div>
                            </div>
                        </div>
                    </div>
                </main>
            </div>
        </div>
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js" crossorigin="anonymous"></script>
        <script src="js/scripts.js"></script>
     </div>
     </div>
    </body>

</html>

Writing templates/result.html


In [ ]:
# <p>&nbsp</p>

In [17]:
app=FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount('/static', StaticFiles(directory='/content/drive/MyDrive/front-end'), name='static')

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("input.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_topic(request: Request, gen: str = Form(...), lyric: str = Form(...)):
    # mkdlyric = ai_lyrics(gen, lyric)
    result = ai_lyrics(gen, lyric)
    # result = result.split('\n')
    fn= open(r'/content/drive/MyDrive/front-end/lyrictxt.txt').read()
    # result1 = result.replace(',', '<br>')
    # result = result.replace(',','<p>&nbsp;</p>')
    # result = result.replace(',', '<p>')
    # output = processing(result)
     
    
    return templates.TemplateResponse("result.html", {"request": request, 'gen': gen, 'lyric':lyric, 'result':result, 'fn':'/static/lyrictxt.txt'})

In [18]:
!ngrok authtoken **************

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

Public URL: http://0dbb-34-147-49-144.ngrok.io


INFO:     Started server process [96]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     211.178.230.174:0 - "GET / HTTP/1.1" 200 OK
INFO:     211.178.230.174:0 - "GET /styles.css HTTP/1.1" 200 OK
INFO:     211.178.230.174:0 - "GET /js/scripts.js HTTP/1.1" 404 Not Found
INFO:     211.178.230.174:0 - "GET /yg.PNG HTTP/1.1" 200 OK
INFO:     211.178.230.174:0 - "GET /assets/%EB%A1%9C%EA%B3%A0.png HTTP/1.1" 404 Not Found


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


[]
[]
[]
[]
tensor([[42000, 42014, 14712, 16522, 16657, 14366,  8066, 20326,     3]],
       device='cuda:0')
이건 누가 봐도 사랑일 텐데 오늘이야 
 나의 맘을 모두 가져간 그대여 
 내게도 사랑이 온다면 
 어떤 말도 그 어떤 표현도 
 내겐 없었던 거야 
 이젠 그댈 사랑해요 
 더 이상 자신 없어도 
 너무 사랑한다는 건 
 내겐 정말 그대뿐인데 
 지금 이순간이 너무 소중해 
 나의 모든 것을 다주어도 
 모자란 내 사랑만으로 
 채워지지 않을 그대를 
 항상 나는 생각해 
 언제까지나 변함없기만을 
 우리의 사랑을 약속해 
 내가 곁에 있고픈 그대의 마음을 
 지켜주고 싶어 
 언제까지나 영원토록 
 그대만 보는 나의 사랑으로 
 어떤 말도 그 어떤 표현도 
 내겐 없었던 거야 
 이젠 그댈 사랑해요 
 이제 그대 내게 있어줘요 
 나의 마음에 오직 그대만을 
 담아두고 싶어 
 항상 나 생각해 
 언제까지나 변함없기만을 
 우리 사랑을 약속해 
 내가 곁에 있고픈 그대의 마음을 
 지켜주고 싶어 
 언제까지나 영원토록 
 그대만 보는 나의 사랑으로 
 이런 감정은 처음이라 
 알 수 없는 이 느낌 
 내가 사랑할 사람 바로 그대인걸 
 이렇게 난 너만을 사랑해 
 더 이상 자신 없어도 
 너무 사랑한다는 건 
 내겐 정말 그대뿐인데 
 지금 이순간이 너무 소중해 
 나의 모든 것을 다주어도 
 모자란 내 사랑만으로 
 채워지지 않을 그대를 
 항상 나는 생각해 
 언제까지나 변함없기만을 
 우리의 사랑을 약속해 
 내가 곁에 있고픈 그대의 마음을 
 지켜주고 싶어 
 언제까지나 영원토록 
 그대만 보는 나의 사랑으로
0.0
이건 누가 봐도 사랑일 텐데 오늘이야 
 나의 맘을 모두 가져간 그대여 
 내게도 사랑이 온다면 
 어떤 말도 그 어떤 표현도 
 내겐 없었던 거야 
 이젠 그댈 사랑해요 
 더 이상 자신 없어도 
 너무 사랑한다는 건 
 내겐 정말 그대뿐인데

...

Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


[]
[]
[]
[]
tensor([[42000, 42014, 15397, 14366, 14674,  4512,     3]], device='cuda:0')


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


아무래도 사랑인가 봐 하 
 널 처음 만난 
 그 순간 난 느꼈어 
 니가 내 사람이란 걸 
 그래 나도 널 보고 있었지 
 하지만 나는 느낄 수 있었어 
 사랑은 영원할 수 없어 
 
 정말 사랑하나봐 
 널 사랑하는 마음처럼 
 내겐 영원할 순 없지만 
 넌 영원할 순 없잖아 
 그래서 이젠 널 사랑해 
 이 노랠 듣고 있는 
 그대 마음을 이해해 볼게 
 그대는 나의 영원한 그대 
 날 사랑하는 만큼 
 항상 노력한다면 
 내 곁에서 영원히 
 날 지켜줘 내 사랑 언제까지나 
 그댈 위해 
 i just wanna be you just be you 
 날 사랑하는 만큼 
 나를 사랑하는 만큼 
 너를 위해서 계속 지켜줄 거야 
 i just wanna be you just be you 
 이런 내 맘 아는 걸까 
 나를 사랑하는 만큼 
 행복하게 살아줘 
 날 사랑해줘 날 사랑해줘 날 사랑해줘 
 
 이런 내맘 아는 걸까 
 나를 사랑하는 만큼 
 나를 사랑하는 만큼 
 너를 위해서 계속 지켜줄 거야 
 i just wanna be you just be you 
 날 사랑하는 만큼 
 나를 사랑하는 만큼 
 너를 위해서 계속 지켜줄 거야 
 i just wanna be you just be you 
 날 사랑하는 만큼 
 나를 사랑하는 만큼 
 나를 사랑하는 만큼 
 너를 위해서 계속 지켜줄 거야 
 i just wanna be you just be you 
 날 사랑하는 만큼 
 나를 사랑하는 만큼 
 널 위해서 계속 지켜줄 거야 
 i just wanna be you just be you 
 날 사랑하는 만큼 
 나를 사랑하는 만큼 
 나를 사랑하는 만큼 
 너를 위해서 계속 지켜줄 거야 
 i just wanna be you just be you
0.27495908346972175
아무래도 사랑인가 봐 하 
 널 처음 만난 
 그 순간 난 느꼈어 
 니가 내 사람이란 걸 
 그래 나도 널 보고 있었지 
 하지만 나는 느낄 수 있었어 
 사랑은 영원할

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [96]


In [ ]:
#    1 # app.mount("/public", StaticFiles(directory=os.path.join(BASE_DIR, "static/public")), name="static")
# ----> 2 BASE_
# width="600px" height="500px" 